In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import json
import urllib.request
import pandas as pd
from pathlib import Path
import json
import seaborn as sns

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML

import plotly.express as px
import folium
import plotly.graph_objects as go



url="https://api.covid19india.org/data.json"
json_url = urllib.request.urlopen(url)
data = json.loads(json_url.read())
df = pd.DataFrame(data['cases_time_series']) 

import plotly.express as px
df1 = pd.DataFrame(data['statewise']) 

data_classification= pd.read_csv(r"D:\Desktop\ca3\dataa.csv")
data_classification

data_clustering= pd.read_csv(r"D:\Desktop\ca3\IndividualDetails.csv")
#data_clustering




In [24]:
data_classification1= pd.read_csv(r"D:\Desktop\ca3\dataa.csv")
data_classification1.drop(data_classification1.columns[[0,1,2,5,6,7,8,9,10,11,12,13,14,15,16]], axis = 1,inplace = True)
data = data_classification1.dropna()

<html>
<style>
</style>
<table><tr><td>
<img src="i4.png"></td>
</tr>
</table>
</html>



In [25]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<ul style="list-style-type:none;" align=center>'
            '<li><h1>INDIA</h1></li>'
             '<li><h2> Confirmed : '+ df.iloc[-1]['totalconfirmed'] + '</h2></li>'
             '<li><h2> Recovered : '+ df.iloc[-1]['totalrecovered'] + '</h2></li>'
            '<li><h2>  Death: '+ df.iloc[-1]['totaldeceased'] + '</h2></li></ul>'))

In [26]:
#preprocessing for regression and visualization
df1=df1.drop([df1.index[0],df1.index[36], df1.index[37]]) 
df1=df1.sort_values('state')

df['dateymd'] = pd.to_datetime(df['dateymd'])
df['dailyconfirmed']=df['dailyconfirmed'].astype(int)
df['dailydeceased']=df['dailydeceased'].astype(int)
df['dailyrecovered']=df['dailyrecovered'].astype(int)
#print(df.dtypes)
df1['active']=df1['active'].astype(int)
df1['confirmed']=df1['confirmed'].astype(int)
df1['recovered']=df1['recovered'].astype(int)
df1['recovered']=df1['recovered'].astype(int)
df1['deaths']=df1['deaths'].astype(int)
#print(df1.dtypes)

df2=df1[df1.state != 'Total']

latitude=[11.7401,15.9129,28.2180,26.2006,25.0961,30.7333,21.2787,20.1809,28.7041,15.2993,22.2587,29.0588,31.1048,
          33.7782,23.6102,15.3173,10.8505,34.152588,22.9734,19.7515,
         24.6637,25.4670,23.1645,26.1584,20.9517,11.9416,31.1471,27.0238,
          27.5330,11.1271,18.1124,23.9408,26.8467,30.0668,22.9868 ]
longitude=[92.6586, 79.7400,94.7278,92.9376,85.3131,76.7794,81.8661,73.0169,77.1025,74.1240,71.1924,76.0856,77.1734,
          76.5762,85.2799,75.7139,76.2711,77.577049,78.6569,75.7139,93.9063,91.3662,92.9376,94.5624,85.0985,
          79.8083,75.3412,74.2179,88.5122,78.6569,79.0193,91.9882,80.9462,79.0193,87.8550]
#df3=df2.head(12)
df1['latitude']=latitude
df1['longitude']=longitude





In [43]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1>Top 10 states with highest active cases</h1></li></ui>'))



<html>
<style>
</style>
<table><tr><td>
<img src="top.png"></td>
</tr>
</table>
</html>


In [57]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1>Spread of covid19 over states of india</h1><br><br><br></li></ui>'))
#spread of covid19 over states of india
india=folium.Map(location=[20.5937,78.9629],tiles="cartodbpositron",zoom_start=5,max_zoom=6,min_zoom=3)
for i in range(len(df1)):
    folium.Circle(
    location=[df1.iloc[i]['latitude'],df1.iloc[i]['longitude']],
    fill=True,
    radius=(int((np.log(df1.iloc[i,1]+1.00001)+0.2)))*10000,
    fill_color='blue',
    color='red',
    tooltip = "<div style ='margin:0;background-color:black;color:white;'>" +
                    "<h5 style='text-align:center;font-weight:bold'>"+df1.iloc[i]['state']+"</h4>"
                    "<hr style='margin:10px;color:white;'>"+
                    "<ul style='color:white;;list-style-type:circle;align-item=left;padding-left:20px;padding-right=20px'>"+
                        "<li>confirmed: "+str(df1.iloc[i,1]) +"</li>"+
                        "<li>deaths:"+str(df1.iloc[i,2])+"</li>"+
                        "<li>deathrate:"+str(np.round(df1.iloc[i,2]/(df1.iloc[i,1]+1.00001)*100,2)) + "</li>"+
                "</ul></div>",
    ).add_to(india)
india


In [46]:
#grouping weekly 
week=[]
x=len(df)
#print(x)
s=0
k=0
for i in range(0, len(df)):
  s=s + df.iloc[i]['dailyconfirmed']
  k=k+1
  if k==7:
    week.append(s)
    s=0
    k=0
    continue
week=np.array(week).reshape(-1,1)
#print(len(week))
#print(week)


In [47]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<ul style="list-style-type:none;" align=center>'
    '<li><h1><u>WEEKLY PREDICTION OF CONFIRMED CASES</u></h1></li><br><br>'
            '<li><h1>Linear Regression</h1></li></ui>'
    '<html><style></style><table><tr><td><img src="LR2.png"></td></tr></table></html>'
    ))
#LinearRegression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
y=week
X=[x for x in range(len(week))]
X=np.array(X).reshape(-1,1)
reg=LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,  random_state=42) 
#print(X_train, X_test, y_train, y_test)
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)
#print(y_pred)

#plt.scatter(X,y)
#plt.scatter(X_test,y_pred)
#plt.plot(X_test,y_pred)
#plt.title("Linear regression ")
#plt.show()
score_lr=reg.score(X_test,y_test)
#print("INTERCEPT :",reg.intercept_)
#print("COEFFICIENT :",reg.coef_)
#print("SCORE : ",reg.score(X_test,y_test))
#print(score_lr)
#x=input("enter the week no ")
#x = int(x) 
"""
interact(f, x='53');

def f(x):
    return x
reg.predict([[x]])
"""
x=widgets.IntText(
    value=7,
    description='week no:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])
x=widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])

widgets.interact(lambda x:print(reg.predict([[x]])), x=widgets.IntSlider(
    value=53,
    min=53,
    max=100,
    step=1,
    description='WEEK NO:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
))

print("")

""


interactive(children=(IntSlider(value=53, continuous_update=False, description='WEEK NO:', min=53), Output()),…

INTERCEPT : [-49817.73530796]
COEFFICIENT : [[12043.85307022]]
SCORE :  0.5664196922462564


In [38]:
#print("COEFFICIENT :",reg.coef_)

COEFFICIENTS : [[12043.85307022]]


In [39]:
#y_pred=reg.predict(X_test)
#print("SCORE : ",reg.score(X_test,y_test))

0.5664196922462564


In [43]:
#print("50 th week predicted value : ",reg.predict([[50]]))

[[552374.91820287]]


In [31]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1>Polynomial Regression</h1></li></ui>'
     '<html><style></style><table><tr><td><img src="PR2.png"></td></tr></table></html>'
    ))
#PolynomialRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
degree=5
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45,  random_state=42) 
polyreg.fit(X_train,y_train)
#plt.figure()
#plt.scatter(X,y)
#plt.scatter(X_test,polyreg.predict(X_test),color="red")
#plt.title("Polynomial regression with degree "+str(degree))
#plt.show()
score_pl=polyreg.score(X_test,y_test)
#print('SCORE : ',score_pl)
x=widgets.IntText(
    value=7,
    description='week no:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])
x=widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])

widgets.interact(lambda x:print(polyreg.predict([[x]])), x=widgets.IntSlider(
    value=53,
    min=53,
    max=100,
    step=1,
    description='WEEK NO:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
))

print("")


""


interactive(children=(IntSlider(value=53, continuous_update=False, description='WEEK NO:', min=53), Output()),…

In [32]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1>Support Vector Regression</h1></li></ui>'
     '<html><style></style><table><tr><td><img src="SR2.png"></td></tr></table></html>'
    
    ))
#supportVector regression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45,  random_state=42) 
svr_rbf = SVR(kernel='rbf', C=1e6, gamma=0.001)
svr_lin = SVR(kernel='linear', C=1e4)
svr_poly = SVR(kernel='poly', C=1e4, degree=2)
y_rbf = svr_rbf.fit(X_train, y_train.ravel()).predict(X_test)
y_lin = svr_lin.fit(X_train, y_train.ravel()).predict(X_test)
y_poly = svr_poly.fit(X_train, y_train.ravel()).predict(X_test)
import pylab as pl
#pl.scatter(X, y, c='k', label='data')
#plt.scatter(X_test, y_rbf, c='r', label='RBF model')
#plt.plot(X_test, y_lin, c='y', label='Linear model')
#plt.scatter(X_test, y_poly, c='b', label='Polynomial model')
#plt.xlabel('data')
#plt.ylabel('target')
#plt.title('Support Vector Regression')
#plt.legend()
#plt.show()
score_sv=svr_rbf.score(X_test,y_test)
#print('SCORE : ',score_sv)
x=widgets.IntText(
    value=7,
    description='week no:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])
x=widgets.Text(
    value='Hello World',
    placeholder='Type something',
    description='String:',
    disabled=False
)
#x = int(x)
#reg.predict([[x]])

widgets.interact(lambda x:print(svr_rbf.predict([[x]])), x=widgets.IntSlider(
    value=53,
    min=53,
    max=100,
    step=1,
    description='WEEK NO:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
))

print("")



""


interactive(children=(IntSlider(value=53, continuous_update=False, description='WEEK NO:', min=53), Output()),…

In [48]:
#prepocessing for clustering
data_clustering.drop(data_clustering.columns[[0,1,2,4,5,6,7,8,9,10,11]], axis = 1,inplace = True) 
data_clustering = data_clustering.dropna()
data_clustering=data_clustering.astype(int)
x=np.array(data_clustering.iloc[:,0]).reshape(-1,1)
#print(x)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [49]:
#preprocessing for classification
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
data_classification.drop(data_classification.columns[[0,1,2,3,4,5,6,7,8,10]], axis = 1,inplace = True) 
nan_value = float("NaN")
data_classification.replace(nan_value," ", inplace=True)
data_classification['symptom1']= label_encoder.fit_transform(data_classification['symptom1']) 
data_classification['symptom2']= label_encoder.fit_transform(data_classification['symptom2'])
data_classification['symptom3']= label_encoder.fit_transform(data_classification['symptom3'])
data_classification['symptom4']= label_encoder.fit_transform(data_classification['symptom4'])
data_classification['symptom5']= label_encoder.fit_transform(data_classification['symptom5'])
data_classification['symptom6']= label_encoder.fit_transform(data_classification['symptom6']) 
'''
print(data_classification['symptom1'].unique())
print(data_classification['symptom2'].unique())
print(data_classification['symptom3'].unique())
print(data_classification['symptom4'].unique())
print(data_classification['symptom5'].unique())
print(data_classification['symptom6'].unique())
print(data_classification['death'].unique())
'''
X=data_classification.iloc[:,[1,2,3,4,5,6]].values
#print(X)
y=data_classification.iloc[:,[0]].values
#print(y)

IndexError: index 7 is out of bounds for axis 0 with size 7

In [52]:
 from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1><u>AGE AND GENDER BASED CLUSTERING</u></h1></li></ui>'
    '<br><br><br><li><h1>K-means Clustering</h1></li><br><br>'))

<html>
<style>
</style>
<table><tr><td>
<img src="Age.png"></td>
</tr>
</table>
</html>


In [56]:
from IPython.core.display import display, HTML
display(HTML(
            
            '<br><br><ul style="list-style-type:none;" align=center>'
            '<li><h1><u>CLASSIFICATION BASED ON SYMPTOMS</u></h1></li></ui>'
            '<br><br><br><li><h1>K-Nearest Neighbour </h1></li></ui>'
    ))
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.40, random_state=42) 

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train.ravel())
y_pred = classifier.predict(X_test)
#print('Score :',classifier.score(X_test,y_test))

#print('Confusion matrix :')
#print(confusion_matrix(y_test, y_pred))
#print('Classification Report:')
#target_names = ['Death', 'Recover']
#print(classification_report(y_test, y_pred, target_names=target_names))


<html>
<style>
</style>
<table><tr><td>
<img src="knnreport.png"></td>
</tr>
</table>
</html>



In [36]:

x1=widgets.Dropdown(
    options=[('Fever', 10), ('High fever', 14), ('Throat pain', 3),('Cough', 4), ('Vomiting', 5), ('Fatigue', 8),
             ('Sore Throat',7),('Not Applicable',0)],
    value=0,
    description='Symptom 1:',
)
x2=widgets.Dropdown(
    options=[('Cough', 32), ('Vomiting', 34), ('Fever', 6),('Joint pain', 16), ('Runny nose', 25), ('Abdominal pain', 1),
             ('Diarrhea',10),('Pneumonia',23),('Respiratory distress',24),('Not Applicable',0)],
    value=0,
    description='Symptom 2:',
)
x3=widgets.Dropdown(
    options=[('Cold', 24), ('Fatigue', 25), ('Difficulty in breathing', 6),('Shortness of breath', 19),
             ('Pneumonia', 5),('Not Applicable',0)],
    value=0,
    description='Symptom 3:',
)
x4=widgets.Dropdown(
    options=[('Nausea', 15), ('Breathlessness', 14), ('Vomiting', 12), ('Diarrhea', 2),('Dyspnea',3),
             ('Difficulty in breathing',13),('Not Applicable',0)],
    value=0,
    description='Symptom 4:',
)
x5=widgets.Dropdown(
    options=[('Loss of Appetite', 2),('Fatigue', 5),
             ('Shortness of breath',3),('Not Applicable',0)],
    value=0,
    description='Symptom 5:',
)
x6=widgets.Dropdown(
    options=[('Joint pain', 3), ('Breathlessness', 2), ('Diarrhea', 1),
             ('Not Applicable',0)],
    value=0,
    description='Symptom 6:',
)

x1


Dropdown(description='Symptom 1:', index=7, options=(('Fever', 10), ('High fever', 14), ('Throat pain', 3), ('…

In [27]:
x2

Dropdown(description='Symptom 2:', index=9, options=(('Cough', 32), ('Vomiting', 34), ('Fever', 6), ('Joint pa…

In [28]:
x3

Dropdown(description='Symptom 3:', index=5, options=(('Cold', 24), ('Fatigue', 25), ('Difficulty in breathing'…

In [29]:
x4

Dropdown(description='Symptom 4:', index=6, options=(('Nausea', 15), ('Breathlessness', 14), ('Vomiting', 12),…

In [125]:
x5

Dropdown(description='Symptom 5:', index=3, options=(('Loss of Appetite', 2), ('Fatigue', 5), ('Shortness of b…

In [126]:
x6

Dropdown(description='Symptom 6:', index=3, options=(('Joint pain', 3), ('Breathlessness', 2), ('Diarrhea', 1)…

In [42]:

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import preprocessing

import sklearn.metrics as sm
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report

#%matplotlib inline
#label_encoder = preprocessing.LabelEncoder()
#data['gender'] = label_encoder.fit_transform(data['gender'])
kmeans = KMeans(n_clusters = 3)
#kmeans.fit(data)
#print(kmeans.labels_)
#plt.scatter(data.age, data.gender, c = kmeans.labels_, cmap = 'rainbow')
#plt.xlabel("Age")
#plt.ylabel("Gender")


#print("Centroids : ")
#print(kmeans.cluster_centers_)



In [37]:
from IPython.display import display

button1 = widgets.Button(description="SHOW")

output1 = widgets.Output()

display(button1,output1)
def on_button_clickeda(b):
    with output1:
        predicted= classifier.predict([[x1.value,x2.value,x3.value,x4.value,x5.value,x6.value]])
        if predicted==1:
            print("DEATH")
        else:
            print("RECOVER")

button1.on_click(on_button_clickeda)

Button(description='SHOW', style=ButtonStyle())

Output()

In [37]:
#RandomForestClassifier
from IPython.core.display import display, HTML
display(HTML(
            
            '<ul style="list-style-type:none;" align=center>'
            '<li><h1>RandomForestClassifier</h1></li></ui>'
    ))
from sklearn.ensemble import RandomForestClassifier
#X_train, X_test, y_train, y_test = train_test_split( 
 #            X, y, test_size = 0.40, random_state=42) 
cls = RandomForestClassifier(n_estimators=10)
cls.fit(X_train, y_train.ravel())
y_pred = cls.predict(X_test)



#print('SCORE :',cls.score(X_test,y_test))
#print('confusion_matrix :')
#print(confusion_matrix(y_test, y_pred))
#print('Classification Report:')
#target_names = ['Death', 'Recover']
#print(classification_report(y_test, y_pred, target_names=target_names))



<html>
<style>
</style>
<table><tr><td>
<img src="RF1.png"></td>
</tr>
</table>
</html>



In [254]:
x1

Dropdown(description='Symptom 1:', index=7, options=(('Fever', 10), ('High fever', 14), ('Throat pain', 3), ('…

In [256]:
x2


Dropdown(description='Symptom 2:', index=9, options=(('Cough', 32), ('Vomiting', 34), ('Fever', 6), ('Joint pa…

In [ ]:
x3

In [ ]:
x4

In [ ]:
x5

In [ ]:
x6

In [40]:

from IPython.display import display

button2 = widgets.Button(description="SHOW")

output2 = widgets.Output()

display(button2,output2)
def on_button_clickedb(b):
    with output2:
        predicted= cls.predict([[x1.value,x2.value,x3.value,x4.value,x5.value,x6.value]])
        if predicted==1:
            print("DEATH")
        else:
            print("RECOVER")

button2.on_click(on_button_clickedb)

Button(description='SHOW', style=ButtonStyle())

Output()

In [39]:

#NaiveBayes
from IPython.core.display import display, HTML
display(HTML(
            
            '<ul style="list-style-type:none;" align=center>'
            '<li><h1>NaiveBayes Classifier</h1></li></ui>'
    ))
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train.ravel()).predict(X_test)
#print("Number of mislabeled points out of a total %d points : %d"
# % (X_test.shape[0], (y_test != y_pred).sum()))
y_pred = gnb.predict(X_test)


#print('SCORE :',gnb.score(X_test,y_test))
#print('confusion_matrix :')
#print(confusion_matrix(y_test, y_pred))
#print('Classification Report:')
#target_names = ['Death', 'Recover']
#print(classification_report(y_test, y_pred, target_names=target_names))
#print('confusion_matrix :')
#print(confusion_matrix(y_test, y_pred))

<html>
<style>
</style>
<table><tr><td>
<img src="NB1.png"></td>
</tr>
</table>
</html>



In [259]:
x1

Dropdown(description='Symptom 1:', index=7, options=(('Fever', 10), ('High fever', 14), ('Throat pain', 3), ('…

In [ ]:
x2

In [ ]:
x3

In [ ]:
x4

In [ ]:
x5

In [260]:
x6

Dropdown(description='Symptom 6:', index=3, options=(('Joint pain', 3), ('Breathlessness', 2), ('Diarrhea', 1)…

In [274]:
from IPython.display import display
from IPython.display import clear_output
button3 = widgets.Button(description="SHOW3")

output = widgets.Output()

display(button3,output)
def on_button_clickedc(b):
    with output:
        predicted= gnb.predict([[x1.value,x2.value,x3.value,x4.value,x5.value,x6.value]])
        if predicted==1:
            print("DEATH")
        else:
            print("RECOVER")

button3.on_click(on_button_clickedc)

Button(description='SHOW3', style=ButtonStyle())

Output()